In [2]:
######## INPUT PARAMETERS ###################
film_id = '96'
#############################################


# запускаем Спарк 

import os
import sys

os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
    
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [3]:
spark

In [4]:
sc

<SparkContext master=yarn appName=pyspark-shell>

In [5]:
# смотрим на данные
!hdfs dfs -head  /labs/laba01/ml-100k/u.data
# user id	item id	rating	timestamp

196	242	3	881250949
186	302	3	891717742
22	377	1	878887116
244	51	2	880606923
166	346	1	886397596
298	474	4	884182806
115	265	2	881171488
253	465	5	891628467
305	451	3	886324817
6	86	3	883603013
62	257	2	879372434
286	1014	5	879781125
200	222	5	876042340
210	40	3	891035994
224	29	3	888104457
303	785	3	879485318
122	387	5	879270459
194	274	2	879539794
291	1042	4	874834944
234	1184	2	892079237
119	392	4	886176814
167	486	4	892738452
299	144	4	877881320
291	118	2	874833878
308	1	4	887736532
95	546	2	879196566
38	95	5	892430094
102	768	2	883748450
63	277	4	875747401
160	234	5	876861185
50	246	3	877052329
301	98	4	882075827
225	193	4	879539727
290	88	4	880731963
97	194	3	884238860
157	274	4	886890835
181	1081	1	878962623
278	603	5	891295330
276	796	1	874791932
7	32	4	891350932
10	16	4	877888877
284	304	4	885329322
201	979	2	884114233
276	564	3	874791805
287	327	5	875333916
246	201	5	884921594
242	1137	5	879741196
249	241	5	879641194
99	4	5	886519097
178	332	3	882823437
251	100	4	886271884
8

In [6]:
# читаем данные, сплитим по табу, отбираем нужное
rdd = sc.textFile('/labs/laba01/ml-100k/u.data')\
        .map(lambda x: x.split('\t'))\
        .map(lambda x: (x[2], x[1]))
rdd.take(5)

[('3', '242'), ('3', '302'), ('1', '377'), ('2', '51'), ('1', '346')]

In [7]:
# фильтруем, считаем количество оценок, сортируем
hist_film = list(dict(sorted(rdd.filter(lambda x: x[1] == film_id).countByKey().items())).values())
hist_film

[6, 20, 43, 123, 103]

In [8]:
# считаем количество оценок, сотрируем
hist_all = list(dict(sorted(rdd.countByKey().items())).values())
hist_all

[6110, 11370, 27145, 34174, 21201]

In [10]:
jsonka = {'hist_film':hist_film,'hist_all':hist_all}
jsonka

{'hist_film': [6, 20, 43, 123, 103],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [11]:
# записываем json в файл
import json
with open('lab01.json', 'w') as fp: 
    json.dump(jsonka, fp)

In [12]:
# смотрим что получилось
! cat lab01.json

{"hist_film": [6, 20, 43, 123, 103], "hist_all": [6110, 11370, 27145, 34174, 21201]}